#### Import library and establish connection to database

In [1]:
# Import library
import ibm_db

# Define credentials
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB" # "db" or DATABASE
dsn_hostname = "dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net" # "hostname" or HOSTNAME
dsn_port = "50000" # "port" or PORT
dsn_protocol = "TCPIP" # PROTOCOL=TCPIP
dsn_uid = "Hidden" # "username" or UID
dsn_pwd = "Hidden" # "password" or PWD

#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#Create database connection
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database!")

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database!


#### Create Table in Database

In [2]:
# Construct the Create Table DDL statement
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

# Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn,createQuery)

#### Insert data into the table

In [3]:
# Construct the query
insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

# Execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [4]:
# Use a single query to insert the remaining two rows of data
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

# Execute the statement
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

#### Query data in the table

In [5]:
# Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

# Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

# Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [6]:
# Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
    print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [7]:
# Write and execute an update statement that changes the Rav's CITY to MOOSETOWN
updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)

#### Retrieve data into Pandas

In [8]:
import pandas
import ibm_db_dbi

In [9]:
# Connection for pandas
pconn = ibm_db_dbi.Connection(conn)

# Query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

# Retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

# See how many rows and columns are in the dataframe
pdf.shape

# Print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


#### Finally, close the connection

In [10]:
ibm_db.close(conn)

True